In [1]:
mnist_results_paths = ['../output/natural-mnist-results.json',
                       '../output/ablation-mnist-results.json',
                       '../output/ablation-mnist-results2.json',
                       '../output/ablation-mnist-results-spsa.json'] 
cifar10_results_paths = ['../output/natural-cifar10-results.json',
                         '../output/natural-cifar10-results2.json',
                         '../output/ablation-cifar10-results.json',
                         '../output/ablation-cifar10-results2.json',
                         '../output/natural-cifar10-results4.json',
                         '../output/ablation-cifar10-results4.json']

In [2]:
import pandas as pd
import numpy as np
from collections import OrderedDict
import re
import os
from scipy import stats

In [3]:
import json

def read_json(path):
    rows = []
    with open(path) as f:
        for line in f:
            if line.strip():
                row = json.loads(line[:-1])
                if 'attack' in row:
                    row['attack_type'] = row['attack']
                    del row['attack']
                rows.append(row)
    df = pd.DataFrame(rows)
    if 'eps' in df.columns:
        df = df.drop_duplicates(['model_path', 'attack_type', 'eps'], keep='last')
    return df

def read_jsons(paths):
    dfs = [read_json(path) for path in paths if os.path.exists(path)]
    return pd.concat(dfs, sort=False).reset_index()

In [4]:
def process(df):
    df['attack_type'] = df.apply(
        lambda r: ('%s(eps=%.1f)' % (r['attack_type'], r['eps'])
                   if 'eps' in r else r['attack_type']), axis=1)
    df['model_name'] = df.model_path.apply(os.path.basename)
    counts = df.groupby(['model_name', 'attack_type']).agg({'accuracies': len})
    duplicates = counts[counts.accuracies >= 2]
    if len(duplicates) > 0:
        print(duplicates.reset_index().values)
    assert len(duplicates) == 0, 'Please remove duplicates'
    return df

# MNIST results

In [5]:
results = process(read_jsons(mnist_results_paths))

In [64]:
def compute_accuracy_and_ttest(col):
    if col.name in ('index', 'model_name'):
        return list(col)
    formatted_accuracies = []
    for prev, curr in zip([None] + list(col), list(col)):
        if isinstance(curr, (str, float)):
            formatted_accuracies.append(curr)
        elif isinstance(curr, list):
            curr = np.around(curr, 2)
            acc = np.mean(curr)
            acc = '%.2f' % acc
            if isinstance(prev, list):
                prev = np.around(prev, 2)
                _, p_value = stats.ttest_ind(prev, curr, equal_var = False)
                acc += '*' if p_value < 0.05 else '<'
            else:
                acc += '<'
            formatted_accuracies.append(acc)
        else:
            raise ValueError("Unsupported type: " + str(curr))
    return pd.Series(formatted_accuracies)

In [72]:
def format_metrics(df, formatted_names, model_order, attack_order, values_col):
    # filtering
    df = df[df.model_name.isin(formatted_names) &
            df.attack_type.isin(attack_order)].copy()
    # convert to wide format
    df = df.pivot(index='model_name', values=values_col, columns='attack_type')
    # change column order
    df = (df.reindex(sorted(df.columns, key=attack_order.__getitem__), axis=1)
          .reset_index().fillna('-'))
    df['model_order'] = df.model_name.apply(model_order.__getitem__)
    df = df.sort_values(['model_order']).drop(columns='model_order')
    # compute mean accuracy and p-values
    df = df.apply(compute_accuracy_and_ttest)
    # format model names last
    df['model_name'] = df.model_name.apply(formatted_names.__getitem__)    
    return df

def format_table(df, formated_names, model_order, attack_order):
    df = format_metrics(df, formatted_names, model_order, attack_order, values_col='accuracies')
    latex = df.to_latex()
    latex = latex.replace('*', '$^*$')
    latex = latex.replace('<', '\hspace{0.5em}')
    return df, latex

## Ablation table

In [99]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog (beta=1)'), 
    ('relog-maxout_4.pkl', '+ MaxOut (k=4)'),
    ('relog-minmaxout_2_4.pkl', '+ MinOut (k=2)'),
    ('relog-elliptical-maxout_4.pkl', '+ Elliptical'),
    ('relog-elliptical-maxout_4-max_fit_l1.pkl', '+ MaxFit (L1)'),  
    ('relog-elliptical-maxout_4-bce-max_fit_l1.pkl', '+ BCE training'),  
    ('relog-elliptical-maxout_4-bce-max_fit_l1-overlay.pkl', '+ Negative examples'),  
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [100]:
used_attacks = ['none(eps=nan)', # (natural)
                'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)', 'CW(eps=nan)',
                'BIM(eps=0.3)', 'SPSA(eps=0.3)']
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [101]:
results.attack_type.unique()

array(['none(eps=nan)', 'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)',
       'CW(eps=nan)', 'BIM(eps=0.3)', 'SPSA(eps=0.3)'], dtype=object)

In [102]:
ablation_table, latex = format_table(results, formatted_names, model_order, attack_order)

In [103]:
ablation_table

attack_type,model_name,none(eps=nan),FGM_inf(eps=0.3),FGM_L2(eps=2.0),CW(eps=nan),BIM(eps=0.3),SPSA(eps=0.3)
7,Baseline,0.99<,0.07<,0.69<,0.01<,0.00<,0.07<
6,+ ReLog (beta=1),0.99*,0.30*,0.84*,0.19*,0.01*,0.37*
4,+ MaxOut (k=4),0.99*,0.42*,0.86*,0.23*,0.03*,0.44*
5,+ MinOut (k=2),0.99<,0.66*,0.92*,0.31*,0.16*,0.52*
3,+ Elliptical,0.93*,0.58*,0.57*,0.47*,0.02*,0.22*
2,+ MaxFit (L1),0.99*,0.35*,0.82*,0.19*,0.02<,0.35*
1,+ BCE training,0.98*,0.56*,0.83*,0.26*,0.16*,0.42*
0,+ Negative examples,0.96*,0.24*,0.66*,0.12*,0.00*,0.14*


In [104]:
print(latex)

\begin{tabular}{llllllll}
\toprule
attack\_type &           model\_name & none(eps=nan) & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) & SPSA(eps=0.3) \\
\midrule
7 &             Baseline &         0.99\hspace{0.5em} &            0.07\hspace{0.5em} &           0.69\hspace{0.5em} &       0.01\hspace{0.5em} &        0.00\hspace{0.5em} &         0.07\hspace{0.5em} \\
6 &     + ReLog (beta=1) &         0.99$^*$ &            0.30$^*$ &           0.84$^*$ &       0.19$^*$ &        0.01$^*$ &         0.37$^*$ \\
4 &       + MaxOut (k=4) &         0.99$^*$ &            0.42$^*$ &           0.86$^*$ &       0.23$^*$ &        0.03$^*$ &         0.44$^*$ \\
5 &       + MinOut (k=2) &         0.99\hspace{0.5em} &            0.66$^*$ &           0.92$^*$ &       0.31$^*$ &        0.16$^*$ &         0.52$^*$ \\
3 &         + Elliptical &         0.93$^*$ &            0.58$^*$ &           0.57$^*$ &       0.47$^*$ &        0.02$^*$ &         0.22$^*$ \\
2 &        + MaxFit (L1) &

## Check a few entries

In [79]:
def print_mean_acc(attack, model):
    row = results[(results.attack_type == attack) & (results.model_name == model)]
    print(np.mean(row.accuracies.iloc[0]))

In [80]:
print_mean_acc('FGM_L2(eps=2.0)', 'relog-elliptical-maxout_4-bce-max_fit_l1-overlay.pkl')

0.6648000000000001


In [81]:
print_mean_acc('BIM(eps=0.3)', 'relog-elliptical-maxout_4-max_fit_l1.pkl')

0.0242


In [82]:
print_mean_acc('SPSA(eps=0.3)', 'relu.pkl')

0.07


# CIFAR-10 results

In [119]:
cifar10_results = process(read_jsons(cifar10_results_paths))

In [120]:
cifar10_results.attack_type.unique()

array(['none(eps=nan)', 'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)',
       'BIM(eps=0.3)', 'CW(eps=nan)'], dtype=object)

In [121]:
cifar10_results.model_name.unique()

array(['relu.pkl', 'relog.pkl', 'relog-maxout.pkl', 'relog-minmaxout.pkl',
       'relog-maxout-elliptical-maxfit-bce.pkl',
       'relog-maxout-elliptical-maxfit-bce-overlay.pkl',
       'relog-maxout-elliptical-maxfit.pkl',
       'relog-maxout-elliptical.pkl'], dtype=object)

In [122]:
formatted_names = OrderedDict([
    ('relu.pkl', 'Baseline'),
    ('relog.pkl', '+ ReLog'), 
    ('relog-maxout.pkl', '+ MaxOut (k=2)'),
    ('relog-minmaxout.pkl', '+ MinOut (k=2)'),
    ('relog-maxout-elliptical.pkl', '+ Elliptical'),
    ('relog-maxout-elliptical-maxfit.pkl', '+ MaxFit (L1)'),
    ('relog-maxout-elliptical-maxfit-bce.pkl', '+ BCE training'),
    ('relog-maxout-elliptical-maxfit-bce-overlay.pkl', '+ Negative examples')
])
model_order = {n: i for i, n in enumerate(formatted_names)}

In [123]:
used_attacks = ['none(eps=nan)', # (natural)
                'FGM_inf(eps=0.3)', 'FGM_L2(eps=2.0)', 'CW(eps=nan)',
                'BIM(eps=0.3)']
attack_order = {n: i for i, n in enumerate(used_attacks)}

In [124]:
ablation_table, latex = format_table(cifar10_results, formatted_names, model_order, attack_order)

In [125]:
ablation_table

attack_type,model_name,none(eps=nan),FGM_inf(eps=0.3),FGM_L2(eps=2.0),CW(eps=nan),BIM(eps=0.3)
7,Baseline,0.76<,0.10<,0.42<,0.01<,0.02<
6,+ ReLog,0.74*,0.17*,0.43<,0.03*,0.01*
4,+ MaxOut (k=2),0.69*,0.17<,0.36*,-,0.01<
5,+ MinOut (k=2),0.72*,-,-,-,-
3,+ Elliptical,0.73<,-,-,-,-
2,+ MaxFit (L1),0.78*,-,-,-,-
1,+ BCE training,0.70*,-,-,-,-
0,+ Negative examples,0.67*,0.10<,0.31<,0.05<,0.01<


In [98]:
print(latex)

\begin{tabular}{lllllll}
\toprule
attack\_type &           model\_name & none(eps=nan) & FGM\_inf(eps=0.3) & FGM\_L2(eps=2.0) & CW(eps=nan) & BIM(eps=0.3) \\
\midrule
7 &             Baseline &         0.76\hspace{0.5em} &            0.08\hspace{0.5em} &           0.43\hspace{0.5em} &       0.01\hspace{0.5em} &        0.01\hspace{0.5em} \\
6 &              + ReLog &         0.75\hspace{0.5em} &            0.16$^*$ &           0.45\hspace{0.5em} &       0.03$^*$ &        0.01\hspace{0.5em} \\
4 &       + MaxOut (k=4) &         0.65$^*$ &            0.12$^*$ &           0.31$^*$ &       0.02$^*$ &        0.01$^*$ \\
5 &       + MinOut (k=2) &         0.70$^*$ &            0.24$^*$ &           0.37$^*$ &       0.04$^*$ &        0.01$^*$ \\
3 &         + Elliptical &         0.68$^*$ &            0.12$^*$ &           0.35\hspace{0.5em} &       0.02$^*$ &        0.01\hspace{0.5em} \\
2 &        + MaxFit (L1) &         0.76$^*$ &            0.10$^*$ &           0.41$^*$ &       0.02\hspace{0